#### Entity Creation
This Notebook demonstrates the creation of entities using existing Azure SQL server Types.  Instead of scanning the data sources, it parses a DDL script to find schemas, tables and columns and then creates the necessary entities.

In [3]:

import yaml, requests, json

# Open the config file to read the client details
with open("../config/config.yaml", "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)

client_id = config[0]['client_id']
client_secret = config[0]['client_secret']
tenant_id = config[0]['tenant_id']
purview_account_name = config[0]['purview_account_name']
scope = f"{config[0]['resource']}/.default"
purview_endpoint = f"https://{purview_account_name}.purview.azure.com"
authority = f"https://login.microsoftonline.com/{tenant_id}"

access_token = ""

#### 1. OAuth Login
Perform Authentication using the Microsoft Authentication Library and get a bearer token for subsequent API calls

In [ ]:
from msal import ConfidentialClientApplication

app = ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)
result = app.acquire_token_for_client(scopes=scope)

access_token = result['access_token']

#### 2. Get the sample DDL script
Let's leverage the sql server samples github repo to download the raw script for the contoso data warehouse.  Database samples are located at https://github.com/microsoft/sql-server-samples/tree/master/samples/databases 

In [197]:

import  urllib3
url = "https://raw.githubusercontent.com/microsoft/sql-server-samples/master/samples/databases/contoso-data-warehouse/load-contoso-data-warehouse-to-sql-data-warehouse.sql"

http = urllib3.PoolManager()

req = http.request('GET', url, preload_content=False)

sql = req.data.decode('utf-8')
#sql = sql.replace("GO", "")


sql


'\n-- Use PolyBase to load public data from Azure Blob Storage into the Contoso Retail Data Warehouse schema.\n--\n-- This script:\n-- \n-- 1. Configures PolyBase for loading from a public blob storage container.\n-- 2. Loads the data into columnstore indexes\n-- 3. Performs optimizations after the load is finished.\n--\n-- Before you begin:\n-- To run this tutorial, you need an Azure account that already has a SQL Data Warehouse database.\n-- If you don\'t already have this, see \n-- http://azure.microsoft.com/documentation/articles/sql-data-warehouse-get-started-provision.md \n--\n-- For more explanation about the loading process, this article on azure.microsoft.com \n-- loads a small set of the Contoso data and explains the process in more detail.\n-- http://azure.microsoft.com/documentation/articles/sql-data-warehouse-load-from-azure-blob-storage-with-polybase.md\n\n\n-- Create an external data source\n-- TYPE: HADOOP - PolyBase uses Hadoop APIs to access data in Azure Blob Storage

In [195]:
import sqlparse
from sqlparse.sql import Identifier, IdentifierList, Parenthesis, TokenList
from sqlparse.tokens import Keyword, Token

# Strip the comments and return a formatted string 
formatted = sqlparse.format(sql, encoding='utf-8', strip_comments=True)

# Parse the text and return a tuple of SQL Statements
statements = sqlparse.parse(formatted, encoding='utf-8')

#statements = sqlparse.split(formatted, encoding='utf-8')
#statements


for statement in statements:
    #print(statement.get_type())

    # If it's a CREATE statement, 
    if statement.get_type() == 'CREATE':
        print("CREATE statement found")
    
    tokens = IdentifierList(statement.tokens).get_identifiers()

    for token in tokens:
        #print(token.get_typecast())
        print(f"{token.__class__}:{token.ttype}:{token}")



        #for token in statement.tokens:
            #print(f"{token.__class__}:{token.ttype}:{token}")
        
        #statement_tokens = statement.tokens
        

        # Get a list of tokens for this statement with whitespace and punctuations removed
        #tokens = IdentifierList(statement_tokens).get_identifiers()    

        
        #for token in tokens:
        #    print(f"{token.__class__}:{token.ttype}:{token}")

    



# Iterate through the tokens to find the CREATE TABLE identifier
#for index, token in enumerate(tokens):
#    print(f"{token.__class__}:{token.ttype}:{token}")


    
#     if item.ttype is Keyword and item.value.upper() == 'TABLE':
#         print(item)
#         tbl_keyword_seen = True



#     if isinstance(item, Identifier) and table_name == "":
#         table_name = item.value
#         print(f"identifier table name: {table_name}")

    

#     if isinstance(item, Parenthesis) and parenthesis_group == 0:
#         print("found parenthesis")
#         # print(item.value)
#         #print(item.is_group)
    
#         tokens = item.flatten()

#         curr_col_name = None
#         for token in tokens:

#            # print(f"{token.__class__}:{token.ttype}:{token}")
#             if token.ttype is Name and curr_col_name is None:
#                 #print(f"column name: {token}")
#                 curr_col_name = token
#             elif token.ttype is Name and curr_col_name is not None :
#                 print(f"Column: {curr_col_name} {token}")
#                 curr_col_name = None
        
#         parenthesis_group += 1

    




#parsed[1].get_type()
#list = parsed[0].get_sublists()

#print(parsed[0].get_name())

#statement = sqlparse.sql.Statement(parsed[0])

#statement.tokens

CREATE statement found
<class 'sqlparse.sql.Token'>:Token.Keyword.DDL:CREATE
<class 'sqlparse.sql.Token'>:Token.Keyword:EXTERNAL
<class 'sqlparse.sql.Token'>:Token.Keyword:DATA
<class 'sqlparse.sql.Token'>:Token.Keyword:SOURCE
<class 'sqlparse.sql.Identifier'>:None:AzureStorage_west_public
<class 'sqlparse.sql.Token'>:Token.Keyword.CTE:WITH
<class 'sqlparse.sql.Parenthesis'>:None:(
    TYPE = Hadoop
,   LOCATION = 'wasbs://contosoretaildw-tables@contosoretaildw.blob.core.windows.net/'
)
<class 'sqlparse.sql.Token'>:Token.Punctuation:;
CREATE statement found
<class 'sqlparse.sql.Token'>:Token.Keyword.DDL:CREATE
<class 'sqlparse.sql.Token'>:Token.Keyword:EXTERNAL
<class 'sqlparse.sql.Token'>:Token.Keyword:FILE
<class 'sqlparse.sql.Identifier'>:None:FORMAT TextFileFormat
<class 'sqlparse.sql.Token'>:Token.Keyword.CTE:WITH
<class 'sqlparse.sql.Parenthesis'>:None:(   FORMAT_TYPE = DELIMITEDTEXT
,	FORMAT_OPTIONS	(   FIELD_TERMINATOR = '|'
					,	STRING_DELIMITER = ''
					,	DATE_FORMAT		 = '

In [ ]:
server_name = "test-server-name"
server_fqdn = "mssql://test-server-name.database.windows.net"
db_name = "test-database"
schema_name = "dbo"
table_name = "test-table"
col1_name = "test-column-1"
col2_name = "test-column-2"
col3_name = "test-column-3"


In [24]:
import requests
import json

# oauth2 login
url = "https://login.microsoftonline.com/" + tenant_id + "/oauth2/token"

# Login and get token
payload='grant_type=client_credentials&client_id=' + client_id + '&client_secret=' + client_secret + '&resource=' + resource
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
}

response = json.loads(requests.request("POST", url, headers=headers, data=payload).content)

# Set the access token
access_token = response['access_token']


#### Create Server, Database and Schema objects

In [25]:
# Create the Server
url = purview_endpoint + "/api/atlas/v2/entity?api-version=2021-05-01-preview"

payload = json.dumps({
  "referredEntities": {},
  "entity": {
    "typeName": "azure_sql_server",
    "attributes": {
      "name": server_name, 
      "qualifiedName": server_fqdn
    }
  }
})
headers = {
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json'
}

# Make the call to the REST endpoint and get the response
response = json.loads(requests.request("POST", url, headers=headers, data=payload).content)


# Get the generated guid from the returned dictionary
assignments = response['guidAssignments']
entityGuids = []

for item in assignments:
    entityGuids.append(assignments.get(item)) 

server_guid = entityGuids[0]

print("Entity created or updated with guid: " + server_guid)


Entity created or updated with guid: 15d37274-ecb7-4317-80ce-4a60cc891baf


In [26]:
# Create the Database
url = purview_endpoint + "/api/atlas/v2/entity?api-version=2021-05-01-preview"

payload = json.dumps({
  "referredEntities": {},
  "entity": {
    "typeName": "azure_sql_db",
    "attributes": {
      "name":  db_name, 
      "qualifiedName": server_fqdn + "/" + db_name
    }
  }
})
headers = {
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json'
}

# Make the call to the REST endpoint and get the response
response = json.loads(requests.request("POST", url, headers=headers, data=payload).content)


# Get the generated guid from the returned dictionary
assignments = response['guidAssignments']
entityGuids = []

for item in assignments:
    entityGuids.append(assignments.get(item)) 

db_guid = entityGuids[0]

print("Entity created or updated with guid: " + db_guid)

Entity created or updated with guid: ad983e1b-d556-4186-a36a-8b15f7624cb7


In [27]:
# Create the Schema 
url = purview_endpoint + "/api/atlas/v2/entity?api-version=2021-05-01-preview"

payload = json.dumps({
  "referredEntities": {},
  "entity": {
    "typeName": "azure_sql_schema",
    "attributes": {
      "name": schema_name, 
      "qualifiedName": server_fqdn + "/" + db_name + "/" + schema_name
    }
  }
})
headers = {
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json'
}

# Make the call to the REST endpoint and get the response
response = json.loads(requests.request("POST", url, headers=headers, data=payload).content)


# Get the generated guid from the returned dictionary
assignments = response['guidAssignments']
entityGuids = []

for item in assignments:
    entityGuids.append(assignments.get(item)) 

schema_guid = entityGuids[0]

print("Schema created or updated with guid: " + schema_guid)

Schema created or updated with guid: aa96e70d-934d-407a-8c3f-70478be6d722


In [29]:
# Create the Table 
url = purview_endpoint + "/api/atlas/v2/entity?api-version=2021-05-01-preview"

payload = json.dumps({
  "referredEntities": {},
  "entity": {
    "typeName": "azure_sql_table",
    "attributes": {
      "name": table_name, 
      "qualifiedName": server_fqdn + "/" + db_name + "/" + schema_name + "/" + table_name
    }
  }
})
headers = {
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json'
}

# Make the call to the REST endpoint and get the response
response = json.loads(requests.request("POST", url, headers=headers, data=payload).content)


# Get the generated guid from the returned dictionary
assignments = response['guidAssignments']
entityGuids = []

for item in assignments:
    entityGuids.append(assignments.get(item)) 

table_guid = entityGuids[0]

print("Table created or updated with guid: " + table_guid)

Table created or updated with guid: 5aae710c-b970-4d33-9f2a-72f6f6f60000


In [31]:
# For the columns we can create them in bulk using the bulk entity endpoint:  https://learn.microsoft.com/en-us/rest/api/purview/catalogdataplane/entity/create-or-update-entities?tabs=HTTP
url_bulk = purview_endpoint + "/api/atlas/v2/entity/bulk?api-version=2021-05-01-preview"

# Create columns
payload = json.dumps({
  "referredEntities": {},
  "entities": [
    {
      "typeName": "azure_sql_column",
      "attributes": {
        "name": col1_name, 
        "qualifiedName": server_fqdn + "/" + db_name + "/" + schema_name + "/" + table_name + "/" + col1_name,
        "data_type": "nvarchar"
      }
    },
    {
      "typeName": "azure_sql_column",
      "attributes": {
        "name": col2_name, 
        "qualifiedName": server_fqdn + "/" + db_name + "/" + schema_name + "/" + table_name + "/" + col2_name,
        "data_type": "integer"
      }
    },
    {
      "typeName": "azure_sql_column",
      "attributes": {
        "name": col3_name, 
        "qualifiedName": server_fqdn + "/" + db_name + "/" + schema_name + "/" + table_name + "/" + col3_name,
        "data_type": "double"
      }
    }
  ]  
})
headers = {
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json'
}

response = json.loads(requests.request("POST", url_bulk, headers=headers, data=payload).content)

# Get the generated guids from the returned dictionary
assignments = response['guidAssignments']
column_guids = []

for item in assignments:
    column_guids.append(assignments.get(item)) 
    print(assignments.get(item))


print(str(len(column_guids)) + " assets created or updated")

588e92ba-0bff-48b7-8703-602f618c3538
78cf6600-bc0b-4e4f-8840-c04f4633a768
0db12503-589e-4ad1-b251-d34b8b48ab3f
3 assets created or updated


In [1]:
# create a new relationship between the server and the database

url_rel = purview_endpoint + "/api/atlas/v2/relationship?api-version=2021-05-01-preview"

payload = json.dumps({
    "typeName": "azure_sql_server_databases",
        "end1": {
            "typeName": "azure_sql_server",
            "guid": server_guid
        },
        "end2": {
            "typeName": "azure_sql_db",
            "guid": db_guid
        }
    }
)
headers = {
    'Authorization': 'Bearer ' + access_token,
    'Content-Type': 'application/json'
}
response = json.loads(requests.request("POST", url_rel, headers=headers, data=payload).content)
print(response)


NameError: name 'purview_endpoint' is not defined

In [ ]:
# create a new relationship between the database and the schema

url_rel = purview_endpoint + "/api/atlas/v2/relationship?api-version=2021-05-01-preview"

payload = json.dumps({
    "typeName": "azure_sql_db_schemas",
        "end1": {
            "typeName": "azure_sql_db",
            "guid": db_guid
        },
        "end2": {
            "typeName": "azure_sql_schema",
            "guid": schema_guid
        }
    }
)
headers = {
    'Authorization': 'Bearer ' + access_token,
    'Content-Type': 'application/json'
}
response = json.loads(requests.request("POST", url_rel, headers=headers, data=payload).content)
print(response)

In [ ]:
# create a new relationship between the schema and the table

url_rel = purview_endpoint + "/api/atlas/v2/relationship?api-version=2021-05-01-preview"

payload = json.dumps({
    "typeName": "azure_sql_schema_tables",
        "end1": {
            "typeName": "azure_sql_schema",
            "guid": schema_guid
        },
        "end2": {
            "typeName": "azure_sql_table",
            "guid": table_guid
        }
    }
)
headers = {
    'Authorization': 'Bearer ' + access_token,
    'Content-Type': 'application/json'
}
response = json.loads(requests.request("POST", url_rel, headers=headers, data=payload).content)
print(response)

In [32]:
# create a new relationship between the table and the columns

url_rel = purview_endpoint + "/api/atlas/v2/relationship?api-version=2021-05-01-preview"

# iterate through the columns and call the endpoint for each one

x = 0

for column_guid in column_guids:
  payload = json.dumps({
        "typeName": "azure_sql_table_columns",
        "end1": {
            "typeName": "azure_sql_table",
            "guid": table_guid
        },
         "end2": {
            "typeName": "azure_sql_column",
            "guid": column_guid
        }
  })
  headers = {
    'Authorization': 'Bearer ' + access_token,
    'Content-Type': 'application/json'
  }
  response = json.loads(requests.request("POST", url_rel, headers=headers, data=payload).content)
